<a href="https://colab.research.google.com/github/physicaone/loss_IG/blob/master/%5B2105165%5DMake_critical_Ising.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import time
from tqdm import tqdm, tnrange


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
def magnetization(lattice):
    '''
    This is a function that returns absolute of spontaneous magnetization of a spin configuration.
    Not returns magnetization per spin.
    '''
    L = len(lattice)
    m = 0
    for i in range(L):
        for j in range(L):
            m += lattice[i,j]
    
    return abs(m)

In [5]:
def Ising_model(lattice, T, n0, d_delta, mcmax, n_sample):
    '''
    This if a function which conducts computing statistics of 2D Ising lattice. 
    This function returns (ensemble) averaged magnetization per spin, magnetic susceptibility, and 4th cummulant (the Binder cummulant).
    '''
    
    L = len(lattice)
    samples=[]
    
    m = 0
    equil = 0
    count = 0

    while equil <= mcmax:
        if equil < n0:
            for i in range(L):
                for j in range(L):
                    s = lattice[i,j]
                    neigh = lattice[(i+1)%L, j] + lattice[i,(j+1)%L] + lattice[(i-1)%L,j] + lattice[i,(j-1)%L]
                    DE = 2 * s * neigh 

                    if DE < 0:
                        s *= -1 
                    elif np.random.rand() < np.exp(- DE*1/T):
                        s *= -1 
                    lattice[i,j] = s
            equil += 1;

        else:
            x = np.random.randint(L)
            y = np.random.randint(L)
            s = lattice[x,y]  
            neigh = lattice[(x+1)%L, y] + lattice[x,(y+1)%L] + lattice[(x-1)%L,y] + lattice[x,(y-1)%L]
            DE = 2 * s * neigh 

            #accept or reject 
            if DE < 0:
                s *= -1 

            elif np.random.rand() < np.exp(- DE*1/T):
                s *= -1 

            lattice[x,y] = s
            count += 1;
            m = 0;

            if count == n_delta:
                count = 0;

            equil += 1;
            a=lattice.copy()
            samples.append(np.float32(np.reshape(a,(1,16))+1)/2)
    return samples


In [6]:
L=4
lattice1 = - np.ones((L,L))


T =2.3


n0 = 2000
n_delta = 100
mcmax = 2**16+n0-1


In [7]:
lat = Ising_model(lattice1, T, n0, n_delta, mcmax, 2**16)
np.shape(lat)

(65536, 1, 16)

In [8]:
import pickle as pkl
with open('drive/MyDrive/loss_IG/Ising/lr=0.01/4*4_cri_full.pkl', 'wb') as f:
    pkl.dump(lat, f)

In [9]:
np.random.shuffle(lat)
with open('drive/MyDrive/loss_IG/Ising/lr=0.01/4*4_cri_sub.pkl', 'wb') as f:
    pkl.dump(lat[:6000], f)